In [1]:
import tensorflow as tf
import os
import PIL
import numpy as np

# Classification type de roue


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1,2"  # specify which GPU(s) to be used

In [3]:
# constantes

NUM_CALLS = tf.data.experimental.AUTOTUNE

DATA_PATH_ROUES_ENTIERES="/home/cogrannr/roues/MEFRO/Images_Roues/Actuelle/gris_blanc/"


In [4]:
"""
    Fonction custom, car doit être fait en eager execution, sinon c'est un param "none" passé à Image.open
    ouvereture des photos / resize
"""
def open_image(file_name): 
    name = tf.get_static_value(file_name)
    name = name.decode()
    
    image = PIL.Image.open(name)
    
    image = image.resize(IMAGE_SIZE)
    image_arr = np.array(image)

    return image_arr
    
def parse_images(filename,label): # appel "open_image"
    image_arr = tf.py_function(open_image,[filename],tf.float32)
    image_arr = tf.image.convert_image_dtype(image_arr,tf.float32)
    return image_arr,label

def data_augmentation(image,label): # semble pas bien marcher
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_brightness(image, max_delta=1.0/255.0)
    #image = tf.clip_by_value(image, 0.0, 1.0)

    return image,label
    
def reset_shapes(image,label):
    image.set_shape(list(INPUT_IMAGE_SHAPE))
    return image, label

In [ ]:
"""
    chargement des listes de fichiers photo
"""

# load pour classif de type 
def load_data():
    label = 0
    for directory in os.listdir(DATA_PATH_ROUES_ENTIERES):
        file_names = tf.data.Dataset.list_files(DATA_PATH_ROUES_ENTIERES+directory+"/*OK*")
        label_dataset = tf.data.Dataset.from_tensors(label).repeat()
        
        if label == 0:
            data_set = tf.data.Dataset.zip((file_names, label_dataset))
            
        else:
            data = tf.data.Dataset.zip((file_names, label_dataset))
            data_set = data_set.concatenate(data)
    
        label = label+1
        
    tf.random.set_seed(1)
    data_set = data_set.shuffle(DATA_SIZE)
    
    return data_set

# load les donnees des roues avec meme label pour les roues qui se ressemblent beaucoup
def load_data_similar_wheel():
    label = 0
    for directory_list in [["05","07"],["04","08","29"],["11","12"],["77","70"],["09","36"],["13"],["75"]]:
        for directory in directory_list:
            file_names = tf.data.Dataset.list_files(DATA_PATH_ROUES_ENTIERES+directory+"/*OK*")
            label_dataset = tf.data.Dataset.from_tensors(label).repeat()
    
            if label == 0:
                data_set = tf.data.Dataset.zip((file_names, label_dataset))
            else:
                data = tf.data.Dataset.zip((file_names, label_dataset))
                data_set = data_set.concatenate(data)

        label = label+1
        
    tf.random.set_seed(1)
    data_set = data_set.shuffle(DATA_SIZE)

    return data_set


# load pour classif binaire
def load_data_entieres(cover_dir,faulty_dir, num_negative, num_positive): 
    
    file_names_cover = tf.data.Dataset.list_files(cover_dir,shuffle=False)
    labels_0 = tf.data.Dataset.from_tensors(0).repeat()
    
    data_set_cover = tf.data.Dataset.zip((file_names_cover,labels_0))
    data_set_cover = data_set_cover.take(num_positive)
    
    file_names_faulty = tf.data.Dataset.list_files(faulty_dir,shuffle=False)
    labels_1 = tf.data.Dataset.from_tensors(1).repeat()
    
    data_set_faulty = tf.data.Dataset.zip((file_names_faulty,labels_1))
    data_set_faulty = data_set_fault.take(num_negative)

    data_set = data_set_cover.concatenate(data_set_faulty)
  
    
    tf.random.set_seed(1)
    data_set = data_set.shuffle(DATA_SIZE)
    
    return data_set
    
    
""" 
    répartition des fichiers enn ensembles train/valid
"""
def generate_train_valid_test(cover_dir="",faulty_dir="", augment_data=False, etape=1,num_negative=0, num_positive=0): 
    
    if etape == 1:
        data_set = load_data_similar_wheel()
    if etape == 2:
        data_set = load_data_entieres(cover_dir, faulty_dir,num_negative, num_positive)
        
    train_dataset = data_set.take(DATA_TRAIN_SIZE)
    valid_dataset = data_set.skip(DATA_TRAIN_SIZE).take(DATA_VALID_SIZE)
    test_dataset = data_set.skip(DATA_TRAIN_SIZE+DATA_VALID_SIZE).take(DATA_TEST_SIZE)

    datasets_list = []
    
    for data_set in [train_dataset, valid_dataset, test_dataset]:

        if data_set == train_dataset:
            data_set = data_set.map(parse_images, num_parallel_calls=NUM_CALLS)
            if augment_data == True:
                data_set = data_set.map(reset_shapes, num_parallel_calls=NUM_CALLS)
                data_set = data_set.map(data_augmentation, num_parallel_calls=NUM_CALLS)
        else:
            data_set = data_set.map(parse_images, num_parallel_calls=NUM_CALLS)
       
        data_set = data_set.map(reset_shapes, num_parallel_calls=NUM_CALLS)
            
        data_set = data_set.batch(BATCH_SIZE)
        data_set = data_set.prefetch(1)
        
        datasets_list.append(data_set)
        
    return datasets_list

In [ ]:
def confusion_matrix(test_dataset,model, num_positive_class, num_negative_class):
    
    prediction = np.array([])
    labels = tf.constant([], dtype=tf.int32)

    for (data_batch, label_batch) in test_dataset:
        prediction = np.concatenate((prediction, model.predict_on_batch(data_batch)[:,0]))
        labels = tf.concat((labels, label_batch), axis=0)

    confusion_matrix = tf.math.confusion_matrix(
        labels=labels,predictions=tf.convert_to_tensor(prediction),num_classes=2)

    matrix = tf.get_static_value(confusion_matrix)
    matrix = matrix.astype(np.float32)

    matrix[0,:] = matrix[0,:]/num_positive_class
    matrix[1,:] = matrix[1,:]/num_negative_class

    print(confusion_matrix)
    print(matrix)

In [6]:
def init_model(architecture=1):
    
    if architecture == 1:    # arch etape 1 
        model_input = tf.keras.Input(shape=INPUT_IMAGE_SHAPE)
        conv_1 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='SAME',strides=(1,2))(model_input)
        conv_2 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='SAME',strides=(1,1))(conv_1)
        bn_1 = tf.keras.layers.BatchNormalization()(conv_2)
        mxp_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_1)
        
        conv_3 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='SAME',strides=(1,2))(mxp_1)
        conv_4 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='SAME',strides=(1,1))(conv_3)
        bn_2 = tf.keras.layers.BatchNormalization()(conv_4)
        mxp_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_2)
        
        conv_5 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='SAME',strides=(1,2))(mxp_2)
        conv_6 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='SAME',strides=(1,1))(conv_5)
        bn_3 = tf.keras.layers.BatchNormalization()(conv_6)
        mxp_3 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_3)
        
        conv_7 = tf.keras.layers.Conv2D(256,(3,3),activation='relu',padding='SAME',strides=(1,2))(mxp_3)
        conv_8 = tf.keras.layers.Conv2D(256,(3,3),activation='relu',padding='SAME',strides=(1,1))(conv_7)
        bn_4 = tf.keras.layers.BatchNormalization()(conv_8)
        mxp_4 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_4)
        
        flat = tf.keras.layers.Flatten()(mxp_4)
        drop = tf.keras.layers.Dropout(0.5)(flat)
        out = tf.keras.layers.Dense(7,activation="softmax")(drop)
    
        return model
    
    if architecture == 2: # etape 2 : transfert
          
        model = tf.keras.models.load_model("model_v3_entieres_etape_1.h5", compile=False)        
        model.trainable = False
        
        out = tf.keras.layers.Dense(1,activation="sigmoid",name="output_layer")(model.layers[-2].output)
        
    model = tf.keras.Model(inputs=model.input, outputs=out)
    print(model.summary())

    return model

    


In [7]:
def train_model(model,train_dataset,valid_dataset, check_point_name, etape,tensor_board_name=""):    
    call_backs_list = [
        #tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience = 2),
        tf.keras.callbacks.ModelCheckpoint(
            filepath =+check_point_name, monitor ="val_accuracy", save_best_only = True),
    ]
    
    precision_metric = tf.keras.metrics.Precision(thresholds=THRESHOLD_PREDICTION)

    
    if tensor_board_name != "":
        board = tf.keras.callbacks.TensorBoard(log_dir='./tensor_board_'+tensor_board_name)
        call_backs_list.append(board)
   
    optim = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    
    if etape == 1:
        model.compile(loss="sparse_categorical_crossentropy",optimizer=optim,
            metrics=["accuracy"]) 
         
    if etape == 2:
        model.compile(loss='binary_crossentropy',optimizer=optim,
            metrics=[precision_metric,"accuracy"]) 
 
    model.fit(train_dataset,epochs=NUM_EPOCHS,
              validation_data=valid_dataset,callbacks=call_backs_list
             )
    
    return model

 # Etape 1 : classif° Type

In [ ]:
IMAGE_SIZE = (1624,1234)  # Images resize en taille ~ moyenne : moyenne pondérée
INPUT_IMAGE_SHAPE = (1234,1624,3) # shape qui rentre dans le cnn
THRESHOLD_PREDICTION = 0.5

MODEL_NAME = "transfer_learning_step_1.h5"
SAVE_PATH = "./trained_models/transfer_learning/"

DATA_SIZE = int(40000) # taille des données (test/valid/test compris), max : 73184
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)

BATCH_SIZE = 9
LEARNING_RATE = 0.0001
NUM_EPOCHS = 3


In [61]:
mirrored_strategy = tf.distribute.MirroredStrategy() 

with mirrored_strategy.scope():

    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
        augment_data=False, etape=1)
    
    print(train_dataset)
    model = init_model(architecture=1)
    model = train_model(model, train_dataset, valid_dataset,etape=1,
                                  check_point_name=SAVE_PATH+MODEL_NAME, )


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
<PrefetchDataset shapes: ((None, 1234, 1624, 3), (None,)), types: (tf.float32, tf.int32)>


KeyboardInterrupt: 

In [7]:
# test sur defauts fabriqués

train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
    augment_data=False, etape=1)

check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME)

print("evaluating model model : ")
check_point.evaluate(test_dataset, verbose=1)



05
07
04
08
29
11
12
77
70
09
36


KeyboardInterrupt: 

#  Etape 2 : classif° defauts

In [ ]:
DATA_SIZE = int(5524) # taille des données (test/valid/test compris), max : 74533;    5524 : rapport 1:4
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)

NUM_POSITIVE_CLASS_TOTAL = 1381 # constant
NUM_NEGATIVE_CLASS_TOTAL = DATA_SIZE - NUM_POSITIVE_CLASS_TOTAL

NUM_POSITIVE_CLASS = int(NUM_POSITIVE_CLASS_TOTAL*0.2) #276 constant
NUM_NEGATIVE_CLASS = int(NUM_NEGATIVE_CLASS_TOTAL*0.2) # 828 pour 1:4    ; 14630 pour tout 

SAVE_PATH = "./trained_models/transfer_learning/"
MODEL_NAME = "transfer_learning_step2.h5"

LEARNING_RATE = 0.0001
NUM_EPOCHS = 5
BATCH_SIZE = 4


In [8]:
mirrored_strategy = tf.distribute.MirroredStrategy() 

with mirrored_strategy.scope():
    
    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
        cover_dir=DATA_PATH_ROUES_ENTIERES+"*/*OK*", 
        faulty_dir=(DATA_PATH_ROUES_ENTIERES+"*/*DJ*",DATA_PATH_ROUES_ENTIERES+"*/*DG*"),
        augment_data=True, etape=2, num_positive=NUM_POSITIVE_CLASS_TOTAL, num_negative=NUM_NEGATIVE_CLASS_TOTAL)

    model = init_model(architecture=2)
    model = train_model(model, train_dataset, valid_dataset,etape=2,
                                  check_point_name=SAVE_PATH+MODEL_NAME)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


AttributeError: module 'tensorflow_core._api.v2.data' has no attribute 'take'

In [9]:
# test sur defauts fabriqués

train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
        cover_dir=DATA_PATH_ROUES_ENTIERES+"*/*OK*", 
        faulty_dir=(DATA_PATH_ROUES_ENTIERES+"*/*DJ*",DATA_PATH_ROUES_ENTIERES+"*/*DG*"),
        augment_data=True, etape=2, num_positive=NUM_POSITIVE_CLASS_TOTAL, num_negative=NUM_NEGATIVE_CLASS_TOTAL)

check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME)
print("evaluating model model : ")
check_point.evaluate(test_dataset, verbose=1)

confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)

evaluating model model for model v3: 
122/122 [==============================] - 72s 587ms/step - loss: 0.0543 - precision: 1.0000 - accuracy: 0.9963


[0.054304208495983004, 1.0, 0.9963437]